<a href="https://colab.research.google.com/github/mabidnadzif/sistem-rekomendasi/blob/main/Sistem_Rekomendasi_content_based_filtering_0027.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content Based Filtering : Rekomendasi Modul Pembelajaran Dari Program Magang Dan Studi Independen Kampus Merdeka

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/mabidnadzif/sistem-rekomendasi/main/dataperusahaan-0027.csv")
df.head()

,Mitra,Alamat,Modul
0,Mukhamad Abid Nadzif,"Ds. Tosari, Rt. 01 Rw.05, Kec. Brangsong, Kab....",Implementasi Basis Data Web Service Sistem Bas...
1,PT Panji Edukasi Nusantara Apple Developer Aca...,Apple Developer Academy @ UC Universitas Ciput...,Subject Strands The subject strands represent...
2,PT Nurul Fikri Cipta Inovasi,Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tug...,Desain Web Version Control System (VCS) Git/Gi...
3,PT Hacktivate Teknologi Indonesia,"Gedung Aquarius Lt. 1&2, Jl. Sultan Iskandar M...",Memahami serta mengimplementasikan library hin...
4,PT Orbit Ventura Indonesia,"Veteran RI Building 15th Floor Unit Z15-002, P...",Artificial Intelligence Machine Learning Deep...


## 1. Ikhtisar

In [2]:
df.describe()

,Mitra,Alamat,Modul
count,12,12,12
unique,12,12,12
top,Mukhamad Abid Nadzif,"Ds. Tosari, Rt. 01 Rw.05, Kec. Brangsong, Kab....",Implementasi Basis Data Web Service Sistem Bas...
freq,1,1,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Mitra   12 non-null     object
 1   Alamat  12 non-null     object
 2   Modul   12 non-null     object
dtypes: object(3)
memory usage: 416.0+ bytes


## 2. Tampilan Modul Pembelajaran (Sebelum Preprocessing)

In [4]:
def print_description(index):
    example = df[df.index == index][['Modul', 'Mitra','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Mitra:', example[1])
        print('Alamat:', example[2])

In [5]:
print_description(1)

Subject Strands  The subject strands represent categories of content the learners experience during the learning journey  Technical  Logic and Programming Interfaces Development Developer Tools Media Animation and Games Operating Systems Platform Functionalities and Supporting Frameworks  Design  Accessibility Branding Design Fundamentals HIG Basic Concepts Prototyping User-Centered Design Game Design and art direction and HIG Advanced Concepts  Process  CBL Engage CBL Investigate CBL Act CBL Supporting Activities Project Management and Scrum  Business and Marketing  App Business App Marketing Entrepreneurship Legal Guidelines Store Guidelines Store Presence and User Engagement  Professional Skills Collaboration Communication Storytelling Presentations Creative Workflow Employability Personal Growth 
Mitra: PT Panji Edukasi Nusantara Apple Developer Academy 
Alamat: Apple Developer Academy @ UC Universitas Ciputra, Citraland, Made, Sambikerep,Surabaya, 60219


In [9]:
print_description(11)

Framework & Desain Pattern Application Programming Interface Reporting & Documentation Dasar-dasar Pemrograman Web Database & Penerapan Model Source code & Task Management Managemen View & Controller Team Collaboration
Mitra: PT Indi Teknokreasi Internasional
Alamat: Graha INDI, Jalan Pahlawan Revolusi No 1, Lt4 RT.06, RT.11/RW.7, Pd. Bambu, Kec. Duren Sawit, Kota Jakarta Timur, Daerah Khusus Ibukota Jakarta 13430


In [10]:
print_description(0)

Implementasi Basis Data Web Service Sistem Basis Data Pemrograman Web Jaringan Komputer Sistem Operasi Pemrograman Aplikasi Mobile Text Mining 
Mitra: Mukhamad Abid Nadzif
Alamat: Ds. Tosari, Rt. 01 Rw.05, Kec. Brangsong, Kab. Kendal


## 3. Text Preprocessing

In [14]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['modul_pembelajaran_clean'] = df['Modul'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
df.head()

,Mitra,Alamat,Modul,modul_pembelajaran_clean
0,Mukhamad Abid Nadzif,"Ds. Tosari, Rt. 01 Rw.05, Kec. Brangsong, Kab....",Implementasi Basis Data Web Service Sistem Bas...,implementasi basis data web service sistem bas...
1,PT Panji Edukasi Nusantara Apple Developer Aca...,Apple Developer Academy @ UC Universitas Ciput...,Subject Strands The subject strands represent...,subject strands the subject strands represent...
2,PT Nurul Fikri Cipta Inovasi,Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tug...,Desain Web Version Control System (VCS) Git/Gi...,desain web version control system vcs git gi...
3,PT Hacktivate Teknologi Indonesia,"Gedung Aquarius Lt. 1&2, Jl. Sultan Iskandar M...",Memahami serta mengimplementasikan library hin...,memahami serta mengimplementasikan library hin...
4,PT Orbit Ventura Indonesia,"Veteran RI Building 15th Floor Unit Z15-002, P...",Artificial Intelligence Machine Learning Deep...,artificial intelligence machine learning deep...


## 4. Tampilan Modul Pembelajaran (Setelah Preprocessing)

In [19]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['modul_pembelajaran_clean', 'Mitra','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Mitra:', example[1])
        print('Alamat:', example[2])

In [20]:
print_description_clean(1)

subject strands  the subject strands represent categories of content the learners experience during the learning journey  technical  logic and programming interfaces development developer tools media animation and games operating systems platform functionalities and supporting frameworks  design  accessibility branding design fundamentals hig basic concepts prototyping usercentered design game design and art direction and hig advanced concepts  process  cbl engage cbl investigate cbl act cbl supporting activities project management and scrum  business and marketing  app business app marketing entrepreneurship legal guidelines store guidelines store presence and user engagement  professional skills collaboration communication storytelling presentations creative workflow employability personal growth 
Mitra: PT Panji Edukasi Nusantara Apple Developer Academy 
Alamat: Apple Developer Academy @ UC Universitas Ciputra, Citraland, Made, Sambikerep,Surabaya, 60219


In [21]:
print_description_clean(11)

framework  desain pattern application programming interface reporting  documentation dasardasar pemrograman web database  penerapan model source code  task management managemen view  controller team collaboration
Mitra: PT Indi Teknokreasi Internasional
Alamat: Graha INDI, Jalan Pahlawan Revolusi No 1, Lt4 RT.06, RT.11/RW.7, Pd. Bambu, Kec. Duren Sawit, Kota Jakarta Timur, Daerah Khusus Ibukota Jakarta 13430


In [22]:
print_description(0)

Implementasi Basis Data Web Service Sistem Basis Data Pemrograman Web Jaringan Komputer Sistem Operasi Pemrograman Aplikasi Mobile Text Mining 
Mitra: Mukhamad Abid Nadzif
Alamat: Ds. Tosari, Rt. 01 Rw.05, Kec. Brangsong, Kab. Kendal


## 5. TF-IDF & Cosine Similarity

In [23]:
df.set_index('Mitra', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['modul_pembelajaran_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.        , 0.04595314, 0.00486068, 0.02183745,
        0.03870311, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.04324668],
       [0.        , 1.        , 0.0055064 , 0.01345864, 0.0189732 ,
        0.00255164, 0.05616617, 0.00638687, 0.        , 0.        ,
        0.04206856, 0.01546272],
       [0.04595314, 0.0055064 , 1.        , 0.0119919 , 0.        ,
        0.02959789, 0.        , 0.01360197, 0.0113211 , 0.        ,
        0.        , 0.05526849],
       [0.00486068, 0.01345864, 0.0119919 , 1.        , 0.        ,
        0.0488558 , 0.00915303, 0.01586735, 0.02641321, 0.        ,
        0.        , 0.01078151],
       [0.02183745, 0.0189732 , 0.        , 0.        , 1.        ,
        0.00879407, 0.        , 0.01258628, 0.01047573, 0.        ,
        0.        , 0.        ],
       [0.03870311, 0.00255164, 0.02959789, 0.0488558 , 0.00879407,
        1.        , 0.        , 0.09038266, 0.02521117, 0.        ,
        0.        ,

In [24]:
# Set index utama di kolom 'name'
indices = pd.Series(df.index)
indices[:50]

0                                  Mukhamad Abid Nadzif
1     PT Panji Edukasi Nusantara Apple Developer Aca...
2                          PT Nurul Fikri Cipta Inovasi
3                     PT Hacktivate Teknologi Indonesia
4                            PT Orbit Ventura Indonesia
5                                 PT Ozami Inti Sinergi
6                 PT. IlmuKomputerCom Braindevs Sistema
7                              PT Home Credit Indonesia
8                          PT BISA ARTIFISIAL INDONESIA
9                              PT Surya Citra Media Tbk
10                         PT Mojadi Aplikasi Indonesia
11                    PT Indi Teknokreasi Internasional
Name: Mitra, dtype: object

## 6. Modelling

In [37]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_modul = []
    
    # Mengambil nama modul berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 3 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_modul.append(list(df.index)[i])
        
    return recommended_modul

## 7. Prediksi

In [38]:
recommendations('Mukhamad Abid Nadzif')

['PT Nurul Fikri Cipta Inovasi',
 'PT Indi Teknokreasi Internasional',
 'PT Ozami Inti Sinergi']

In [39]:
recommendations("PT Ozami Inti Sinergi")

['PT Home Credit Indonesia',
 'PT Hacktivate Teknologi Indonesia',
 'Mukhamad Abid Nadzif']

In [40]:
recommendations("PT Indi Teknokreasi Internasional")

['PT Nurul Fikri Cipta Inovasi',
 'Mukhamad Abid Nadzif',
 'PT. IlmuKomputerCom Braindevs Sistema']

# Selesai...